<a href="https://colab.research.google.com/github/soccy/Study/blob/main/hongong_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [6]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [7]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [8]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [11]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [14]:
# 상기 결과가 훈련 세트에 과대적합되어있다. 매개변수를 바꿔서 더 좋은 모델을 찾자.
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.03475857, 0.01323986, 0.01262712, 0.01228476, 0.01184177]), 'score_time': array([0.00189734, 0.00179791, 0.00193453, 0.00195289, 0.00176907]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [15]:
# 5-폴드 교차검증을 사용했기에 5개가 나온다.
# 교차검증의 최종 점수는 test_score 에 담긴 5개 점수를 평균하여 얻는다.
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [18]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [19]:
# 만약, 훈련세트를 섞은 후, 10-폴드 교차검증을 수행하려면 다음과 같이한다.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [23]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [24]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [26]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [27]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [29]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [31]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), 'max_depth' : range(5, 20, 1), 'min_samples_split' : range(2, 100, 10)}

In [32]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [33]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [34]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [39]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([3, 2, 0, 5, 2, 5, 7, 3, 0, 4])

In [41]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 109, 101, 104,  98, 107, 104,  76, 105,  98]))

In [42]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.0504118 , 0.88094829, 0.75840038, 0.96095189, 0.11112639,
       0.37647044, 0.33583424, 0.6095622 , 0.52932176, 0.32906569])

In [44]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001), 'max_depth' : randint(20, 50), 'min_samples_split' : randint(2, 25), 'min_samples_leaf' : randint(1, 25)}

In [46]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d1781c6eda0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d1781c6e1a0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d1781c6d3c0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7d1781c6f2b0>},
                   random_state=42)

In [47]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [49]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [51]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
